In [1]:
import os
import sys
import shutil
import re
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import traci
import traci.constants as tc
import sumolib
import xml.etree.ElementTree as ET

# Add the "scripts" directory to sys.path
current_dir = os.path.dirname(os.path.abspath("__file__"))
scripts_dir = os.path.abspath(os.path.join(current_dir, 'scripts'))
sys.path.append(scripts_dir)
import animateGPS
import importlib

FOLDER_NAME = 'unicamp_routine'

In [2]:
sumoBinary = "/usr/bin/sumo-gui"
sumoCmd = [sumoBinary, "-c", "osm.sumocfg"]

In [3]:
# Code of each variable to subscribe:
SPEED = 64
POSITION = 66
ACCELERATION = 114
ANGLE = 67
DEPATURE = 58

In [4]:
def add_xml_child(file_path, parent_tag, child_tag, child_value, replace=True):
    """
    Adds a new child parameter inside a specified parent tag in the XML configuration file.
    If the parent tag does not exist, it creates a new parent tag (<parameter>) with the child.
    It also checks if the child element already exists to prevent duplicates.

    Args:
        file_path (str): Path to the XML configuration file.
        parent_tag (str): The parent tag under which to add the child (e.g., 'input').
        child_tag (str): The child tag to add (e.g., 'additional-files').
        child_value (str): The value to set for the new child tag.
        replace (bool): If True, replaces the existing child tag with the new value.
                        If False, adds another child value.
    Returns:
        bool: True if the addition was successful, False otherwise.
    """
    try:
        # Parse the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Find the parent element by tag
        parent_elem = root.find(parent_tag)
        if parent_elem is None:
            print(f"Parent tag '{parent_tag}' not found. Creating new parent tag.")
            parent_elem = ET.Element(parent_tag)
            root.append(parent_elem)
            print(f"Created new parent tag <{parent_tag}>.")

        # Check if the child element already exists inside the parent element
        existing_child = parent_elem.find(child_tag)
        if existing_child is not None: 
            
            if existing_child.get('value') == child_value:
                print(f"Child <{child_tag}> with value '{child_value}' already exists. Skipping addition.")
                return False
            else:
                if replace:
                    print(f"Child <{child_tag}> already exists. Updating value to '{child_value}'.")
                    existing_child.set('value', child_value)
                else:
                    if child_value in existing_child.get('value').split(', '):
                        print(f"Child <{child_tag}> with value '{child_value}' already exists. Skipping addition.")
                        return False
                    
                    print(f"Child <{child_tag}> already exists. Adding another child with value '{child_value}'.")
                    existing_child.set('value', f'{existing_child.get("value")}, {child_value}')
                
                tree.write(file_path, encoding="UTF-8", xml_declaration=True, method="xml")
                print("XML file updated successfully.")
                return True

        # Create the new child element and set its value
        new_child = ET.Element(child_tag)
        new_child.set('value', child_value)
        print(f"Created <{child_tag}> with value '{child_value}'.")

        # Add the new child to the parent element
        parent_elem.append(new_child)
        print(f"Added <{child_tag}> to <{parent_tag}>.")

        # Write the updated XML to the file
        tree.write(file_path, encoding="UTF-8", xml_declaration=True, method="xml")
        print("XML file updated successfully.")
        return True

    except ET.ParseError as e:
        print(f"XML Parsing error: {e}")
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

In [5]:
def flush_print():
    sys.stdout.write("\r")  # Move the cursor to the beginning of the line
    sys.stdout.write(" " * 50)  # Overwrite with spaces to clear the line
    sys.stdout.write("\r")  # Move back to the beginning again

In [6]:
def get_all_variables(vehIDs, personIDs, routine=None, delta_time=0.05, end_hours=24, useGui=False, convertGeo=True, freeze_traffic_lights=False):
    """
    Function to get all variables from the simulation.
    :param vehIDs: List of vehicle IDs to subscribe to.
    :param personIDs: List of person IDs to subscribe to.
    :param routine: Dictionary with the routine of the vehicles.
    :param useGui: Boolean to use the GUI or not.
    :param convertGeo: Boolean to convert the coordinates from sumo to lat/lon (does not work when using maps from CARLA).
    :param freeze_traffic_lights: If true, traffic lights are green all the time.
    :return: Dictionary with the variables.
    """

    if useGui:
        traci.start(["sumo-gui", "-c", f"{FOLDER_NAME}/osm.sumocfg"])
    else:
        traci.start(["sumo", "-c", f"{FOLDER_NAME}/osm.sumocfg"])
    v_variables = {}
    p_variables = {}

    if end_hours != 0:
        end_time = f'{end_hours * 60 * (60/delta_time)}' # 24 hours
    else:
        end_time = ''
    
    add_xml_child(f'{FOLDER_NAME}/osm.sumocfg', 'time', 'step-length', f'{delta_time}', replace=True)
    add_xml_child(f'{FOLDER_NAME}/osm.sumocfg', 'time', 'end', end_time, replace=True)

    # Get all traffic light IDs
    if freeze_traffic_lights:
        tls_ids = traci.trafficlight.getIDList()

        # Set all traffic lights to constant green
        for tls_id in tls_ids:
            # Get logic info
            logic = traci.trafficlight.getCompleteRedYellowGreenDefinition(tls_id)[0]

            # Create a new green-only phase
            green_state = 'G' * len(logic.phases[0].state)
            green_phase = traci.trafficlight.Phase(duration=9999999, state=green_state)

            # Replace the logic with only this green phase
            new_logic = traci.trafficlight.Logic(
                logic.programID, logic.type, logic.currentPhaseIndex, [green_phase]
            )
            traci.trafficlight.setCompleteRedYellowGreenDefinition(tls_id, new_logic)
    
    # To understand when a car has changed its destionation, we need to know it parked
    if routine:
        status = {}
        for id in routine.keys():
            status[id] = {}
            status[id]['parked'] = False # True if the vehicle is parked
            status[id]['count'] = 0 # Count of number of times a vehicle has parked
            status[id]['stops'] = list(routine[id]['name']) # List of stops for the vehicle
            status[id]['desc'] = f'<home-{status[id]["stops"][1]}>' # Description of the vehicle's status

    time = 0
    arrived = [] # List of vehicles that have arrived at their destination
    while traci.simulation.getMinExpectedNumber() > 0:

        for veh_id in (set(traci.simulation.getDepartedIDList()) & set(vehIDs)): # Subscribe to vehicles that have just departed
            print(f"Vehicle {veh_id} has departed")
            traci.vehicle.subscribe(veh_id, [tc.VAR_POSITION, tc.VAR_SPEED, tc.VAR_ACCELERATION, tc.VAR_ANGLE])
        
        for veh_id in (set(traci.vehicle.getIDList()) & set(vehIDs)): # Checking status of vehicles that are already in the simulation
            if routine:
                is_parked = traci.vehicle.isStoppedParking(veh_id)
                if not status[veh_id]['parked'] and is_parked:  # Vehicle was not parked and now is parked
                    status[veh_id]['parked'] = True
                    status[veh_id]['count'] += 1
                    current_location = status[veh_id]["stops"][status[veh_id]["count"]]
                    next_location = status[veh_id]["stops"][status[veh_id]["count"] + 1] if status[veh_id]["count"] + 1 < len(status[veh_id]["stops"]) else None
                    status[veh_id]['desc'] = f'<{current_location}-{current_location}>'
                    print(f"Vehicle {veh_id} is parked at {current_location}. Count: {status[veh_id]['count']}")

                elif status[veh_id]['parked'] and not is_parked:  # Vehicle was parked and now is not parked
                    status[veh_id]['parked'] = False
                    current_location = status[veh_id]["stops"][status[veh_id]["count"]]
                    next_location = status[veh_id]["stops"][status[veh_id]["count"] + 1] if status[veh_id]["count"] + 1 < len(status[veh_id]["stops"]) else None
                    status[veh_id]['desc'] = f'<{current_location}-{next_location}>'

        for veh_id in (set(traci.simulation.getArrivedIDList()) & set(vehIDs)): # Vehicles that finished their route
            # traci.vehicle.unsubscribe(veh_id)
            if routine:
                print(f"Vehicle {veh_id} has arrived at its destination.")
                status[veh_id]['count'] += 1
                current_location = status[veh_id]["stops"][status[veh_id]["count"]]
                status[veh_id]['desc'] = f'<{current_location}-{current_location}>'
            
            arrived.append(veh_id)
            if (len(arrived) == len(vehIDs)):
                print("All vehicles have arrived at their destination.")
                traci.close()
                return v_variables, p_variables

        results = traci.vehicle.getAllSubscriptionResults().copy()

        for veh_id in results.keys():
            # Converting from x, y sumo coordinates to lat, lon
            if convertGeo:
                x, y = results[veh_id][tc.VAR_POSITION]
                lon, lat = traci.simulation.convertGeo(x, y, fromGeo=False)
                results[veh_id]['longitude'] = lon
                results[veh_id]['latitude'] = lat
                
            if routine:
                results[veh_id]['desc'] = status[veh_id]['desc']

        v_variables[time] = results
        
        time += delta_time
        traci.simulationStep()
        
    traci.close()
    return v_variables, p_variables

In [7]:
def get_departure_arrival(variables, ID):
    # Devolve o tempo de depareture e arrival de um veículo ou pessoa
    departure = 0
    arrival = 0
    departed = False
    arrived = False
    for x in range(len(variables)):
        if ID in variables[x] and variables[x][ID] and not departed:
            departed = True
            departure = x
        elif ID not in variables[x] and departed:
            arrival = x
            arrived = True
            break
    if not arrived:
        arrival = len(variables)
    return departure, arrival

In [8]:
def extract_vtype_ids(route_file_path, sulfix='veh'):
    # sulfix is the suffix of the vehicle ID to identify the vehicles from routines
    root = ET.parse(route_file_path).getroot()
    ids_random = {}
    ids_routine = {}
    for vehicle in (root.findall('vehicle') + root.findall('trip')):
        v_type = re.sub(r'\d+', '', vehicle.get('type').split('_', 1)[1])
        vID = vehicle.get('id')

        if vID.startswith(sulfix):
            if v_type not in ids_routine.keys():
                ids_routine[v_type] = []

            ids_routine[v_type].append(vID)

        else:
            if v_type not in ids_random.keys():
                ids_random[v_type] = []

            ids_random[v_type].append(vID)

    return ids_routine, ids_random

In [9]:
def plot_df(ids, df, sensors, normalize_time=True, time_window=[], use_lat_lon=True, save_path=None):

    fig, axes = plt.subplots(nrows=len(ids), ncols=2, figsize=(15, 5 * len(ids)))
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    df["speed"] = df["speed"].values.astype(float)
    df["acceleration"] = df["acceleration"].values.astype(float)
    df["angle"] = df["angle"].values.astype(float)
    # Shift the time so that it starts at zero
    if normalize_time:
        df["timestamp"] = df["timestamp"] - df["timestamp"].min()
    
    # Limit the time window if specified
    if time_window:
        df = df[(df["timestamp"] >= time_window[0]) & (df["timestamp"] <= time_window[1])]

    if use_lat_lon:
        lat = df["latitude"].values
        lon = df["longitude"].values
    else:
        lat = df["y_pos"].values
        lon = df["x_pos"].values

    for i, veh_id in enumerate(ids):
        # Plot each selected sensor in its own subplot
        plot_line = i * 2
        for sensor in sensors:
            if sensor == 'gps':
                scatter = axes[plot_line + 1].scatter(lon, lat, c=df["timestamp"], cmap='viridis', label="Position (colored by time)", s=df["timestamp"] * 0.05)
                cbar = fig.colorbar(scatter, ax=axes[plot_line + 1])
                cbar.set_label("timestamp")
                axes[plot_line + 1].set_ylim(lat.min() - 0.001, lat.max() + 0.001)
                axes[plot_line + 1].set_xlim(lon.min() - 0.001, lon.max() + 0.001)
                axes[plot_line + 1].set_xlabel("x position")
                axes[plot_line + 1].set_ylabel("y position")
                axes[plot_line + 1].set_title(f"Vehicle position over time")
            else:
                axes[plot_line].plot(df["timestamp"], df[sensor], label=sensor.capitalize())
                axes[plot_line].set_xlim(df["timestamp"].min(), df["timestamp"].max())

        axes[plot_line].set_ylabel("Speed (m/s) / Acceleration (m/s²)")
        axes[plot_line].set_xlabel("Simulation time (s)")
        axes[plot_line].set_title(f"Vehicle speed and acceleration over time")
        axes[plot_line].legend()

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()

    fig.clf()

In [10]:
def save_data(veh_variables, data_folder_name, vehIDs, delta_time, file_names_postfix = '', new_dir = False, type_ids_random = None, verify=True, type_ids_routine=None, use_desc=True, use_lat_lon=True, speed_threshold=6, acc_threshold=6, derivative_threshold=3):

    if new_dir:
        if not os.path.exists(f'{FOLDER_NAME}/{data_folder_name}'):
            os.mkdir(f'{FOLDER_NAME}/{data_folder_name}')
        else:
            shutil.rmtree(f'{FOLDER_NAME}/{data_folder_name}')
            os.mkdir(f'{FOLDER_NAME}/{data_folder_name}')

    for timestep, data in veh_variables.items():
        for vehID, vehData in data.items():

            if use_lat_lon:
                columns = 'timestamp,latitude,longitude,speed,acc,angle,acc_diff,gyro_z'
            else:
                columns = 'timestamp,x_pos,y_pos,speed,acc,angle,acc_diff,gyro_z'

            path = f'{FOLDER_NAME}/{data_folder_name}/{vehID}{file_names_postfix}.csv'
            nolabel_path = f'{FOLDER_NAME}/{data_folder_name}/{vehID}{file_names_postfix}_nolabel.csv'
            if use_desc:
                if not os.path.exists(path):

                    with open(path, 'w') as f:
                        f.write(f'{columns},desc\n')
            
            if not os.path.exists(nolabel_path):
                with open(nolabel_path, 'w') as f:
                    f.write(f'{columns}\n')
                    
            write_speed = vehData[SPEED]
            write_angle = vehData[ANGLE]
            write_acc = vehData[ACCELERATION]
            write_x = vehData[POSITION][0]
            write_y = vehData[POSITION][1]

            if verify:
                derivative_threshold
                try: 
                    derivative_speed = (veh_variables[timestep][vehID][SPEED] - veh_variables[timestep-derivative_threshold][vehID][SPEED]) / derivative_threshold
                    derivative_acceleration = (veh_variables[timestep][vehID][ACCELERATION] - veh_variables[timestep-derivative_threshold][vehID][ACCELERATION]) / derivative_threshold
                except:
                    derivative_speed = 0
                    derivative_acceleration = 0
                    
                # Making verification to ensure there are no outliers
                if derivative_speed > speed_threshold or derivative_speed < -speed_threshold:
                    last_speed = veh_variables[timestep-delta_time][vehID][SPEED]
                    print(f'Vehicle {vehID} at timestep {timestep} had a speed of {vehData[SPEED]}, it was changed to {last_speed}')
                    veh_variables[timestep][vehID][SPEED] = last_speed
                    write_speed = last_speed

                if derivative_acceleration > acc_threshold or derivative_acceleration < -acc_threshold:
                    last_acc = veh_variables[timestep - delta_time][vehID][ACCELERATION]
                    print(f'Vehicle {vehID} at timestep {timestep} had an acceleration of {vehData[ACCELERATION]}, it was changed to {last_acc}')
                    veh_variables[timestep][vehID][ACCELERATION] = last_acc
                    write_acc = last_acc
                
                if vehData[ANGLE] < 0 or vehData[ANGLE] > 360:
                    last_angle = veh_variables[timestep - delta_time][vehID][ANGLE]
                    print(f'Vehicle {vehID} at timestep {timestep} had an angle of {vehData[ANGLE]}, it was changed to {last_angle}')
                    veh_variables[timestep][vehID][ANGLE] = last_angle
                    write_angle = last_angle

            if write_speed < -100: # Invalid values
                write_speed = 0
                print(f'Vehicle {vehID} at timestep {timestep} had a INVALID speed of {vehData[SPEED]}, it was changed to 0')
            if write_acc < -100 or write_acc > 100: # Invalid values
                write_acc = 0
                print(f'Vehicle {vehID} at timestep {timestep} had a INVALID acceleration of {vehData[ACCELERATION]}, it was changed to 0')
            
            try:
                acc_diff = np.abs((veh_variables[timestep][vehID][ACCELERATION] -
                                veh_variables[timestep - delta_time][vehID][ACCELERATION]) / delta_time)

                current_angle = np.radians(veh_variables[timestep][vehID][ANGLE])
                previous_angle = np.radians(veh_variables[timestep - delta_time][vehID][ANGLE])

                # Proper angle wrapping to handle transitions like 359° -> 0°
                angle_diff = np.arctan2(np.sin(current_angle - previous_angle),
                                        np.cos(current_angle - previous_angle))

                gyroscope_z = angle_diff / delta_time  # radians/second
            except:
                acc_diff = np.abs(write_acc)
                gyroscope_z = 0.0

            if use_lat_lon:
                line = f'{timestep},{vehData["latitude"]},{vehData["longitude"]},{write_speed},{write_acc},{write_angle},{acc_diff},{gyroscope_z}'
            else:
                line = f'{timestep},{write_x},{write_y},{write_speed},{write_acc},{write_angle},{acc_diff},{gyroscope_z}'

            if use_desc:
                with open(path, 'a') as f:
                    f.write(f'{line},{vehData["desc"]}\n')
            
            with open(nolabel_path, 'a') as f:
                f.write(f'{line}\n')
        
    # Creating the labels file
    if use_desc:
        with open(f'{FOLDER_NAME}/{data_folder_name}/labels.csv', 'w') as f:
            f.write('ID,Type\n')
            found = False
            for i in range(len(vehIDs)):
            
                if type_ids_random and not found:
                    for key, value in type_ids_random.items():
                        if vehIDs[i] in str(value):
                            f.write(f'{vehIDs[i]},{key}\n')
                            found = True
                            break
                    
                if type_ids_routine and not found:
                    for key, value in type_ids_routine.items():
                        if vehIDs[i] in str(value):
                            f.write(f'{vehIDs[i]},{key}\n')
                            break

# Simulating and getting the data

In [13]:
# IDs of vehicles and persons to subscribe
type_ids_routine, type_ids_random = extract_vtype_ids(f'{FOLDER_NAME}/merged.rou.alt.xml', sulfix='pizza')

# Flatten the type_ids_random values into a 1D array
type_ids_random_1d = [item for sublist in type_ids_random.values() for item in sublist]
type_ids_routine_1d = [item for sublist in type_ids_routine.values() for item in sublist]

# WARNING: ALWAYS LET THE LAST VEHICLES BE THE ONES WITH ROUTINES
# IMPORTANT: Make sure the IDs are strings
# vehIDs = [f'{i}' for i in type_ids_random_1d] + list(type_ids_routine.values()) # For all vehicles
vehIDs = type_ids_routine_1d # For only vehicles with routines
personIDs = []

In [14]:
routines = {}
for k, v in type_ids_routine.items():
    for id in v:
        routines[id] = pd.read_csv(f"{FOLDER_NAME}/routines/{id}.csv").set_index('Time')

routines['pizza']

,location,name,x_pos,y_pos
Time,,,,
7,home,home,-22.821710,-47.074029
8,Instituto de Computação,Instituto de Computação,-22.815546,-47.067300
9,home,home,-22.821710,-47.074029
10,Instituto de Filosofia e Ciências Humanas,Instituto de Filosofia e Ciências Humanas,-22.815256,-47.068342
11,home,home,-22.821710,-47.074029
12,Instituto de Geociências,Instituto de Geociências,-22.813205,-47.068773
13,home,home,-22.821710,-47.074029
14,Faculdade de Engenharia Agrícola,Faculdade de Engenharia Agrícola,-22.818826,-47.060573
15,home,home,-22.821710,-47.074029


In [17]:
Hz = 100
delta_time = 1 / Hz
try:
    veh_variables, ped_variables = get_all_variables(vehIDs, personIDs, delta_time=delta_time, routine=routines, useGui=False, convertGeo=True) # Running the simulation
    
except KeyboardInterrupt as e:
    traci.close() # Close the simulation if there is an error
    print(f"An error occurred: {e}")

 Retrying in 1 seconds
***Starting server on port 47191 ***
Loading net-file from 'unicamp_routine/osm.net.xml.gz' ... done (222ms).
Loading additional-files from 'unicamp_routine/park.add.xml' ... done (13ms).
Loading additional-files from 'unicamp_routine/pa_rerouter.xml' ... done (164ms).
Loading route-files incrementally from 'unicamp_routine/merged.rou.alt.xml'
Loading done.
Simulation version 1.24.0 started with time: 0.00.
Child <step-length> with value '0.01' already exists. Skipping addition.
Child <end> already exists. Updating value to '8640000.0'.
XML file updated successfully.
Vehicle pizza has departed
Vehicle pizza is parked at Instituto de Computação. Count: 1


Vehicle pizza is parked at home. Count: 2


Vehicle pizza is parked at Instituto de Filosofia e Ciências Humanas. Count: 3


Vehicle pizza is parked at home. Count: 4


Vehicle pizza is parked at Instituto de Geociências. Count: 5


...

Vehicle pizza is parked at Instituto de Filosofia e Ciências Humanas. Count: 44


Vehicle pizza is parked at home. Count: 45


Vehicle pizza is parked at Instituto de Física Gleb Wataghin. Count: 46


Vehicle pizza is parked at Restaurante Acadêmico. Count: 47


Vehicle pizza has arrived at its destination.
All vehicles have arrived at their destination.
Simulation ended at time: 21962.79.
Reason: TraCI requested termination.
Performance:
 Duration: 1813.88s
 TraCI-Duration: 1495.91s
 Real time factor: 12.1082
 UPS: 4086.983152
Vehicles:
 Inserted: 441
 Running: 3
 Waiting: 0
 Emergency Braking: 59
Statistics (avg of 438):
 RouteLength: 1758.30
 Speed: 12.91
 Duration: 168.82
 WaitingTime: 0.31
 TimeLoss: 43.35
 DepartDelay: 0.00
DijkstraRouter answered 488 queries and explored 473.82 edges on average.
DijkstraRouter spent 0.28s answering queries (0.58ms on average).


In [18]:
os.makedirs(f'{FOLDER_NAME}/data_routine', exist_ok=True)
save_data(veh_variables, "data_routine", vehIDs, type_ids_random, verify=True, type_ids_routine=type_ids_routine, use_lat_lon=True, use_desc=True)

In [16]:
df_teste = pd.read_csv(f'{FOLDER_NAME}/data_routine/veh_segunda.csv', sep=',')

In [17]:
df_teste.head()

,timestamp,latitude,longitude,speed,speed_x,speed_y,acceleration,acceleration_x,acceleration_y,angle,acc_diff,gyroscope_z,desc
0,0.01,-22.822658,-47.073232,0.000,0.000000,-0.000000,0.0,0.000000,-0.000000,317.599761,0.0,0,<home-Ciclo Básico I>
1,0.02,-22.822658,-47.073232,0.026,0.019200,-0.017532,2.6,1.919977,-1.753194,317.599761,2.6,0,<home-Ciclo Básico I>
2,0.03,-22.822658,-47.073232,0.052,0.038400,-0.035064,2.6,1.919977,-1.753194,317.599761,2.6,0,<home-Ciclo Básico I>
3,0.04,-22.822658,-47.073232,0.078,0.057599,-0.052596,2.6,1.919977,-1.753194,317.599761,2.6,0,<home-Ciclo Básico I>
4,0.05,-22.822658,-47.073232,0.104,0.076799,-0.070128,2.6,1.919976,-1.753194,317.599761,2.6,0,<home-Ciclo Básico I>


In [27]:
def plot_routine(df, idx, save_path=None, time_window=None):
    # Filter by time window if specified
    if time_window is not None and len(time_window) == 2:
        df = df[(df['timestamp'] >= time_window[0]) & (df['timestamp'] <= time_window[1])]

    # Create subplots
    fig, axes = plt.subplots(1, 3, figsize=(25, 5))

    # Plot acceleration
    axes[0].plot(df['timestamp'], df['acceleration_x'], label='Acc X')
    axes[0].plot(df['timestamp'], df['acceleration_y'], label='Acc Y')
    axes[0].plot(df['timestamp'], df['acceleration'], label='Acc Magnitude')
    axes[0].set_title("Acceleration over Time")
    axes[0].set_ylabel("Acceleration (m/s²)")
    axes[0].legend()
    axes[0].grid(True)

    # Plot gyroscope
    axes[1].plot(df['timestamp'], df['angle'], label='angle')
    axes[1].set_title("Angle over Time")
    axes[1].set_ylabel("Angle (rad)")
    axes[1].legend()
    axes[1].grid(True)

    # Plot position
    scatter = axes[2].scatter(df['latitude'], df['longitude'], c=df['timestamp'], cmap='viridis', s=10)
    axes[2].set_title("Position (Latitude vs Longitude) colored by Time")
    axes[2].set_xlabel("Longitude")
    axes[2].set_ylabel("Latitude")
    axes[2].grid(True)
    cbar = fig.colorbar(scatter, ax=axes[2])
    cbar.set_label("Time (s)")

    # Set a single title for the entire figure
    fig.suptitle(f"SUMO Simulator data for route {idx}", fontsize=16)

    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()

In [28]:
fname = f'veh_segunda.csv'
df_routine = pd.read_csv(f'{FOLDER_NAME}/data_routine/{fname}')
idx = re.sub(r'\D', '', fname)  # Extract the numeric part from the filename
os.makedirs(f'{FOLDER_NAME}/plot_routine', exist_ok=True)

In [31]:
plot_routine(df_routine, idx, time_window=[0, 5000], save_path=f'{FOLDER_NAME}/plot_routine/{fname[:-4]}.png')

# Experiment 1

In [15]:
FOLDER_NAME = 'uah_map'

In [18]:
experiment1_files = os.listdir(f'{FOLDER_NAME}/experiment1')
delta_time = 0.01 # This delta time has to be the same as the one used in CARLA
end_time = 0 # 0 means no end time, the simulation will run until all vehicles finish their route
with open(f'{FOLDER_NAME}/experiment1/vehicle_ids.txt', 'r') as f:
    vehIDs = f.read().splitlines()


if os.path.exists(f'{FOLDER_NAME}/data_experiment1_sumo'):
    shutil.rmtree(f'{FOLDER_NAME}/data_experiment1_sumo')
os.makedirs(f'{FOLDER_NAME}/data_experiment1_sumo', exist_ok=True)

for i in range(len(vehIDs)):

    add_xml_child(f'{FOLDER_NAME}/osm.sumocfg', 'input', 'route-files', f'experiment1/{vehIDs[i]}.rou.xml', replace=True)
    personIDs = []

    try:
        veh_variables, ped_variables = get_all_variables(vehIDs, personIDs, delta_time=delta_time, routine=None, end_hours=end_time, useGui=False, convertGeo=False, freeze_traffic_lights=True) # Running the simulation
        save_data(veh_variables, "data_experiment1_sumo", [vehIDs[i]], delta_time, new_dir=False, verify=True, use_lat_lon=False, use_desc=False)
        
    except KeyboardInterrupt as e:
        traci.close() # Close the simulation if there is an error
        print(f"An error occurred: {e}")

Child <route-files> with value 'experiment1/veh0_normal.rou.xml' already exists. Skipping addition.
 Retrying in 1 seconds
***Starting server on port 43753 ***
Loading net-file from 'uah_map/osm.net.xml.gz' ... done (23ms).
Loading route-files incrementally from 'uah_map/experiment1/veh0_normal.rou.xml'
Loading done.
Simulation version 1.23.1 started with time: 0.00.
Child <step-length> with value '0.01' already exists. Skipping addition.
Child <end> already exists. Updating value to ''.
XML file updated successfully.
Vehicle veh0_normal has departed
Simulation ended at time: 504.18.
Reason: TraCI requested termination.
Performance:
 Duration: 12.93s
 TraCI-Duration: 12.09s
 Real time factor: 38.984
 UPS: 3898.322122
Vehicles:
 Inserted: 1
 Running: 0
 Waiting: 0
Statistics (avg of 1):
 RouteLength: 10556.89
 Speed: 20.94
 Duration: 504.17
 WaitingTime: 0.25
 TimeLoss: 6.71
 DepartDelay: 0.00
DijkstraRouter answered 1 queries and explored 39.00 edges on average.
DijkstraRouter spent 0.

In [14]:
def plot_df_exp1(df, idx, save_path = None):
    # Create subplots
    fig, axes = plt.subplots(1, 3, figsize=(25, 5))

    # Plot acceleration
    axes[0].plot(df['timestamp'], df['acc_x'], label='Acc X')
    axes[0].plot(df['timestamp'], df['acc_y'], label='Acc Y')
    axes[0].set_title("Acceleration over Time")
    axes[0].set_ylabel("Acceleration (m/s²)")
    axes[0].legend()
    axes[0].grid(True)

    # Plot gyroscope
    axes[1].plot(df['timestamp'], df['gyro_z'], label='Gyro Z')
    axes[1].set_title("Gyroscope over Time")
    axes[1].set_ylabel("Angular Velocity (rad/s)")
    axes[1].legend()
    axes[1].grid(True)

    # Plot position
    scatter = axes[2].scatter(df['x_pos'], df['y_pos'], c=df['timestamp'], cmap='viridis', s=10)
    axes[2].set_title("Position (Latitude vs Longitude) colored by Time")
    axes[2].set_xlabel("Longitude")
    axes[2].set_ylabel("Latitude")
    axes[2].grid(True)
    cbar = fig.colorbar(scatter, ax=axes[2])
    cbar.set_label("Time (s)")

    # Set a single title for the entire figure
    fig.suptitle(f"SUMO Simulator data for route {idx}", fontsize=16)

    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()

Saving the plots

In [ ]:
if os.path.exists(f'{FOLDER_NAME}/plot_experiment1_sumo'):
    shutil.rmtree(f'{FOLDER_NAME}/plot_experiment1_sumo')
os.makedirs(f'{FOLDER_NAME}/plot_experiment1_sumo', exist_ok=True)

for f in os.listdir(f'{FOLDER_NAME}/data_experiment1_sumo'):
    df = pd.read_csv(f'{FOLDER_NAME}/data_experiment1_sumo/{f}')
    idx = re.sub(r'\D', '', f)  # Extract the numeric part from the filename
    plot_df_exp1(df, idx, save_path=f'{FOLDER_NAME}/plot_experiment1_sumo/{f[:-4]}.png')

In [ ]:
os.listdir(f'{FOLDER_NAME}/data_experiment1_sumo')

## Routine data

In [ ]:
FOLDER_NAME = "town01"

In [ ]:
id = 'veh_carla0_nolabel'
df = pd.read_csv(f'{FOLDER_NAME}/data_routine/{id}.csv')
plot_df([id], df, ['acceleration_x', 'gps'], normalize_time=True, time_window=[0, 300], use_lat_lon=False)

In [ ]:
importlib.reload(animateGPS)
veh_id = 'veh_carla0_nolabel'
animateGPS.animate_gps(
    veh_id=veh_id,
    folder_path= os.path.join(os.getcwd(), f'{FOLDER_NAME}/data_routine'),
    frame_step=5,             # optional
    time_window=[0, 50000],     # optional
    start_time=7,
    show_desc=False,        # optional (True to show the description of the vehicle if it has)
    save_path=os.path.join(os.getcwd(), f'{FOLDER_NAME}/{veh_id}_animation.mp4')  # <<< Save instead of showing live
)

## UFSCar test

In [ ]:
FOLDER_NAME = "ufscar"

In [ ]:
id = '99'
df = pd.read_csv(f'{FOLDER_NAME}/data_dirty/{id}.csv')
plot_df(['99'], df, ['acceleration_x'], normalize_time=False, time_window=[])

To solve this problem, one idea is to identify places where the changes are too abrupt using an approximate derivative and substitute it for the previus value

In [ ]:
save_data(copy.deepcopy(veh_variables), "data", vehIDs, type_ids_random, type_ids_routine=None, verify=True, speed_threshold=5, acc_threshold=4, derivative_threshold=2)

In [ ]:
id = '99'
df = pd.read_csv(f'{FOLDER_NAME}/data/{id}.csv')
plot_df(['99'], df, ['acceleration_x'], normalize_time=False, time_window=[])

## Now lets boxplot the data to see how separate they are

In [ ]:
def get_whole_data(foldername, type_ids_random):
    df = {}
    for cls in type_ids_random.keys():
        cls_df = pd.DataFrame()
        for vehID in type_ids_random[cls]:
            csv_path = f'{foldername}/{vehID}.csv'
            df_temp = pd.read_csv(csv_path)
            cls_df = pd.concat([cls_df, df_temp], axis=0)
        df[cls] = cls_df    
    return df

In [ ]:
def boxplots(df, sensors, showfliers=True, names=None, classes=None):
    fig, axes = plt.subplots(nrows=int(np.ceil(len(sensors)/2)), ncols=2, figsize=(15, 5 * int(np.ceil(len(sensors)/2))))
    axes = axes.flatten()
    cls = list(df.keys())

    for j in range(len(sensors)):

        if classes:
            axes[j].boxplot([df[cls[i]][sensors[j]] for i in range(len(df))], tick_labels=classes, showfliers=showfliers)
        else:
            axes[j].boxplot([df[cls[i]][sensors[j]] for i in range(len(df))], tick_labels=cls, showfliers=showfliers)

        if names:
            axes[j].set_ylabel(f'{names[j].capitalize()}')
            axes[j].set_title(f'Boxplot de {names[j].split(" ")[0]}')
        else:
            axes[j].set_title(f'Boxplot de {sensors[j].capitalize()}')
            axes[j].set_ylabel(f'{sensors[j].capitalize()}')

    for j in range(len(sensors), len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

Cleaned data

In [ ]:
df = get_whole_data(f'{FOLDER_NAME}/data', type_ids_random)
boxplots(df, sensors=['acceleration', 'speed', 'angle', 'acc_diff'], showfliers=True, names=['aceleração (m/s²)', 'velocidade (m/s)', 'ângulo (graus)', 'acc_diff (m/s²)'], classes=['cauteloso', 'normal', 'agressivo', 'super agressivo'])

Dirty data

In [ ]:
df_dirty = get_whole_data(f'{FOLDER_NAME}/data_dirty', type_ids_random)
boxplots(df_dirty)

## Example of time series for each class

In [ ]:
def show_examples(data_folder, ids, sensors, window=None, plot_en=False):
    fig, axes = plt.subplots(nrows=len(ids)//2, ncols=2, figsize=(15, 2 * len(ids)), sharex=False)
    axes = axes.flatten()

    y_min, y_max = float('inf'), float('-inf')

    # Primeira passagem para determinar os limites de y
    for i in range(len(ids)):
        id = list(ids.values())[i]
        time_series = pd.read_csv(f'{data_folder}/{id}.csv')
        for sensor in sensors:
            y_min = min(y_min, time_series[sensor][:window].min())
            y_max = max(y_max, time_series[sensor][:window].max())

    # Segunda passagem para plotar os dados com limites de y compartilhados
    for i in range(len(ids)):
        id = list(ids.values())[i]
        cls = list(ids.keys())[i]
        time_series = pd.read_csv(f'{data_folder}/{id}.csv')
        for sensor in sensors:
            axes[i].plot(time_series['time'], time_series[sensor], label=sensor.capitalize())

            if not plot_en:
                if cls == 'cautious':
                    cls = 'cauteloso'
                elif cls == 'aggressive':
                    cls = 'agressivo'
                elif cls == 'super_aggressive':
                    cls = 'super agressivo'

            if plot_en:
                axes[i].set_title(f"{cls.capitalize()} vehicle over time")
            else:
                axes[i].set_title(f"Veículo {cls} ao longo do tempo")
            
            
            if window:
                axes[i].set_xlim([time_series['time'].min(), time_series['time'].min() + window])
            else:
                axes[i].set_xlim([time_series['time'].min(), time_series['time'].max()])
            axes[i].set_ylim([y_min - 1, y_max + 1])

    if plot_en:
        for ax in axes:
            ax.set_xlabel("Simulation step")
            ax.set_ylabel("Speed (m/s) / Acceleration (m/s²)")
            ax.legend(['Speed', 'Acceleration'])

    else:
        for ax in axes:
            ax.set_xlabel("Passo de simulação")
            ax.set_ylabel("Velocidade (m/s) / Aceleração (m/s²)")
            ax.legend(['Velocidade', 'Aceleração'])

    plt.tight_layout()
    plt.show()

In [ ]:
show_examples(f"{FOLDER_NAME}/data", {x: type_ids_random[x][np.random.randint(0, len(type_ids_random[x]))] for x in type_ids_random.keys()}, ['speed', 'acceleration'], window=400, plot_en=True)

### Here we can clearly see that even though the means of the values are really similar to each other, the frequency of change of acceleration, for example, is a lot different. Maybe we can use that to better separate the data

## Boxplot for two classes

In [ ]:
type_ids_routine_simple, type_ids_random_simple = extract_vtype_ids('unicamp')
df = get_whole_data(f'unicamp/data_simple', type_ids_random_simple)
boxplots(df)

## Decomposing sensors into X and Y axis

The idea here is to try to make it easier to separate the data by having more features

In [ ]:
# TODO